# Making Pandas DataFrames from API Requests

In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/


### Get Data from API


In [2]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

In [27]:
api="https://api.nobelprize.org/2.1/nobelPrizes"
nobel_response = requests.get(api, params=payload)

In [28]:
nobel_response.text

'{"nobelPrizes":[{"awardYear":"1901","category":{"en":"Chemistry","no":"Kjemi","se":"Kemi"},"categoryFullName":{"en":"The Nobel Prize in Chemistry","no":"Nobelprisen i kjemi","se":"Nobelpriset i kemi"},"dateAwarded":"1901-11-12","prizeAmount":150782,"prizeAmountAdjusted":8722510,"links":[{"rel":"nobelPrize","href":"https://api.nobelprize.org/2/nobelPrize/che/1901","action":"GET","types":"application/json"}],"laureates":[{"id":"160","knownName":{"en":"Jacobus H. van \'t Hoff"},"fullName":{"en":"Jacobus Henricus van \'t Hoff"},"portion":"1","sortOrder":"1","motivation":{"en":"in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions","se":"såsom ett erkännande av den utomordentliga förtjänst han inlagt genom upptäckten av lagarna för den kemiska dynamiken och för det osmotiska trycket i lösningar"},"links":[{"rel":"laureate","href":"https://api.nobelprize.org/2/laureate/160","action":"GET","types":"app

In [8]:
import requests

url = "https://realtor.p.rapidapi.com/locations/v2/auto-complete"

querystring = {"input":"new york","limit":"10"}

headers = {
	"X-RapidAPI-Key": "96e7c1d34cmsh78649e4f03123a2p12713cjsn975ba60afde1",
	"X-RapidAPI-Host": "realtor.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'message': 'You are not subscribed to this API.'}


Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response

We need to check the structures of the response data to know where our data is.


In [30]:
nobel_json = nobel_response.json()
nobel_json.keys()

dict_keys(['nobelPrizes', 'meta', 'links'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:


In [31]:
nobel_json['nobelPrizes']

[{'awardYear': '1901',
  'category': {'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'},
  'categoryFullName': {'en': 'The Nobel Prize in Chemistry',
   'no': 'Nobelprisen i kjemi',
   'se': 'Nobelpriset i kemi'},
  'dateAwarded': '1901-11-12',
  'prizeAmount': 150782,
  'prizeAmountAdjusted': 8722510,
  'links': [{'rel': 'nobelPrize',
    'href': 'https://api.nobelprize.org/2/nobelPrize/che/1901',
    'action': 'GET',
    'types': 'application/json'}],
  'laureates': [{'id': '160',
    'knownName': {'en': "Jacobus H. van 't Hoff"},
    'fullName': {'en': "Jacobus Henricus van 't Hoff"},
    'portion': '1',
    'sortOrder': '1',
    'motivation': {'en': 'in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions',
     'se': 'såsom ett erkännande av den utomordentliga förtjänst han inlagt genom upptäckten av lagarna för den kemiska dynamiken och för det osmotiska trycket i lösningar'},
    'links': [{'r

Each element in the JSON array `features` is a row of data for our dataframe.


In [33]:
type(nobel_json['nobelPrizes'])

list

Your data will be different depending on the date you run this.


In [42]:
type(nobel_json['nobelPrizes'][0]["laureates"][0]["id"])

str

In [43]:
nobel_dataframe=pd.DataFrame(nobel_json['nobelPrizes'])
nobel_dataframe.head()
nobel_dataframe.to_csv("noble_prize.csv")

In [45]:
nobel_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   awardYear            25 non-null     object
 1   category             25 non-null     object
 2   categoryFullName     25 non-null     object
 3   dateAwarded          21 non-null     object
 4   prizeAmount          25 non-null     int64 
 5   prizeAmountAdjusted  25 non-null     int64 
 6   links                25 non-null     object
 7   laureates            25 non-null     object
dtypes: int64(2), object(6)
memory usage: 1.7+ KB


In [46]:
nobel_dataframe.describe()

,prizeAmount,prizeAmountAdjusted
count,25.000000,2.500000e+01
mean,142587.000000,8.105418e+06
std,4388.290622,3.481829e+05
min,138089.000000,7.753291e+06
25%,140859.000000,7.908819e+06
50%,141358.000000,7.936836e+06
75%,141847.000000,8.205634e+06
max,150782.000000,8.722510e+06


In [48]:
nobel_dataframe["awardYear"].unique()

array(['1901', '1902', '1903', '1904', '1905'], dtype=object)

In [51]:
nobel_dataframe["awardYear"].value_counts()

awardYear
1901    5
1902    5
1903    5
1904    5
1905    5
Name: count, dtype: int64

In [54]:
nobel_dataframe["awardYear"].value_counts().index

Index(['1901', '1902', '1903', '1904', '1905'], dtype='object', name='awardYear')

### Convert to DataFrame

We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.


In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,0.57,"8 km SW of Volcano, Hawaii",1692143828870,1692145618780,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73528487,",",hv,",",origin,phase-data,",12.0,NaN,0.07,85.0,md,earthquake,"M 0.6 - 8 km SW of Volcano, Hawaii"
1,1.49,"15 km WSW of Watonga, Oklahoma",1692143693530,1692205506728,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ok2023pyta,",",ok,",",origin,phase-data,",43.0,0.188063,0.27,97.0,ml,earthquake,"M 1.5 - 15 km WSW of Watonga, Oklahoma"
2,0.89,"8 km SW of Volcano, Hawaii",1692143488740,1692145885540,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73528472,",",hv,",",origin,phase-data,",12.0,NaN,0.08,152.0,md,earthquake,"M 0.9 - 8 km SW of Volcano, Hawaii"
3,0.56,"8 km SW of Volcano, Hawaii",1692143304920,1692146314540,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73528462,",",hv,",",origin,phase-data,",15.0,NaN,0.07,83.0,md,earthquake,"M 0.6 - 8 km SW of Volcano, Hawaii"
4,4.10,"46 km NNE of San Pedro de Atacama, Chile",1692143291492,1692145893040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000knkf,",",us,",",origin,phase-data,",13.0,0.444000,0.40,78.0,mb,earthquake,"M 4.1 - 46 km NNE of San Pedro de Atacama, Chile"


### (Optional) Write Data to CSV


In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>
